# The Problem: What countries will have increased listening time based on the amount of user engagement

In [74]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [75]:
music_data = pd.read_csv('../data/clean_data/country_listen_time.csv')
music_data.head(1)

,age,minutes_streamed_per_day,repeat_song_rate,discover_weekly_engagement,number_of_songs_liked
0,34,295,16.74,47.42,138


## Preprocessing

In [76]:
# Extract features and target
features = ['age','repeat_song_rate','discover_weekly_engagement','number_of_songs_liked']
target_col = ['minutes_streamed_per_day']

x_raw = music_data[features].values
y_raw = music_data[target_col].values.reshape(-1,1)

print(f"\nDataset size: {len(music_data)} individual users")
print(f"Input features: {len(features)} numerical features")
print(f"Target: Individual listening minutes")

print(f"\nFeature statistics:")
for i, feature in enumerate(features):
    values = x_raw[:, i]
    print(f"{feature}:")
    print(f" Range: {values.min():.1f} = {values.max():.1f}")
    print(f" Mean: {values.mean():.1f}")
    print(f" Standard Deviation: {values.std():.1f}")


"""
How target statistics will help to verify if we are getting correct numbers

1. Range (Min - Max): Data Spread Analysis
    * Scale awareness: 600 minutes = 10 hours (huge daily listening!)
    * Outlier detection: Are there unrealistic values?
    * Network design: Large ranges need careful normalization
    * Activation choice: ReLU works well with positive ranges

2. Mean: Central Tendency

    * Baseline prediction: Random guessing would predict the mean
    * Normalization center: StandardScaler centers data around 0
    * Reality check: Does average make sense? (5 hours/day is reasonable)
    * R2 calculation: R2 compares predictions to mean baseline

3. Standard Deviation: Variability
    * Learning difficulty: High std = more patterns to learn
    * Normalization scaling: StandardScaler uses std for scaling
    * Prediction quality: Good models should predict within 1-2 std of actual
    * Feature importance: Target variation drives what features matter    
"""

print(f"\nTarget Statistics")
print(f"Minutes Streamed Per Day")
print(f" Range: {y_raw.min():1f} - {y_raw.max():.1f}")   # Data spread
print(f" Mean: {y_raw.mean():.1f}")                      # Central tendency
print(f" Standard Deviation {y_raw.std():.1f}")          # Variability



Dataset size: 5000 individual users
Input features: 4 numerical features
Target: Individual listening minutes

Feature statistics:
age:
 Range: 13.0 = 60.0
 Mean: 36.7
 Standard Deviation: 13.8
repeat_song_rate:
 Range: 5.0 = 80.0
 Mean: 42.4
 Standard Deviation: 21.4
discover_weekly_engagement:
 Range: 10.0 = 90.0
 Mean: 50.3
 Standard Deviation: 23.2
number_of_songs_liked:
 Range: 1.0 = 500.0
 Mean: 253.5
 Standard Deviation: 146.4

Target Statistics
Minutes Streamed Per Day
 Range: 10.000000 - 600.0
 Mean: 309.2
 Standard Deviation 172.0


In [77]:
# Normalize all features
scaler_features = StandardScaler()
x = scaler_features.fit_transform(x_raw)

# Normalize target values
scaler_target = StandardScaler()
y = scaler_target.fit_transform(y_raw)

print(f"\nAfter normalization")
print(f"Features - Mean: {x.mean():.3f}, Standard Deviation: {x.std():.3f}")
print(f"Target - Mean: {y.mean():.3f}, Standard Deviation: {y.std():.3f}")

# Split data into train/val/test (60%/20%/20%)
x_train, x_temp, y_train, y_temp = train_test_split(x,y, test_size=0.4, random_state=42)
x_val,x_test,y_val,y_test = train_test_split(x_temp,y_temp, test_size=0.5, random_state=42)

print(f"\nData Splits:")
print(f"Training set: {x_train.shape[0]} samples")
print(f"Validation set: {x_val.shape[0]} samples")
print(f"Test set: {x_test.shape[0]} samples")


After normalization
Features - Mean: 0.000, Standard Deviation: 1.000
Target - Mean: 0.000, Standard Deviation: 1.000

Data Splits:
Training set: 3000 samples
Validation set: 1000 samples
Test set: 1000 samples


## Neural Network Class

In [78]:
class RegressionNeuralNetwork:
    """
    2-layer Neural Network specialized for Regression tasks
    - Continuous output with linear activation
    - Mean Squared Error loss function
    - R² score evaluation metric
    """
    
    def __init__(self, input_size, hidden_size, output_size=1):
        """
        Initialize regression neural network
        
        Args:
            input_size: Number of input features
            hidden_size: Number of hidden layer neurons
            output_size: Number of continuous outputs (usually 1)
        """
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        
        # Xavier initialization
        self.W1 = np.random.randn(input_size, hidden_size) / np.sqrt(input_size)
        self.b1 = np.zeros((1, hidden_size))
        
        self.W2 = np.random.randn(hidden_size, output_size) / np.sqrt(hidden_size)
        self.b2 = np.zeros((1, output_size))
        
    def relu(self, x):
        """ReLU activation: f(x) = max(0, x)"""
        return np.maximum(0, x)
    
    def relu_derivative(self, x):
        """ReLU derivative: f'(x) = 1 if x > 0, else 0"""
        return (x > 0).astype(float)
    
    def forward_pass(self, X):
        """
        Forward propagation for regression:
        z1 = X @ W1 + b1
        a1 = ReLU(z1)
        z2 = a1 @ W2 + b2  (Linear output, no activation)
        """
        # Hidden layer
        self.z1 = X @ self.W1 + self.b1
        self.a1 = self.relu(self.z1)
        
        # Output layer (linear for regression)
        self.z2 = self.a1 @ self.W2 + self.b2
        
        return self.z2
    
    def mse_loss(self, y_true, y_pred):
        """
        Mean Squared Error loss for regression:
        L = Σ(y_true - y_pred)² / (2m)
        """
        m = y_true.shape[0]
        mse = np.sum((y_true - y_pred) ** 2) / (2 * m)
        return mse
    
    def backward_pass(self, X, y_true):
        """
        Backpropagation for regression:
        
        Output layer (MSE derivative):
        dL/dz2 = (y_pred - y_true) / m
        
        Hidden layer:
        dL/da1 = dL/dz2 @ W2^T
        dL/dz1 = dL/da1 * ReLU'(z1)
        """
        m = X.shape[0]
        
        # Output layer gradients (MSE derivative)
        dz2 = (self.z2 - y_true) / m
        dW2 = self.a1.T @ dz2
        db2 = np.sum(dz2, axis=0, keepdims=True)
        
        # Hidden layer gradients
        da1 = dz2 @ self.W2.T
        dz1 = da1 * self.relu_derivative(self.z1)
        dW1 = X.T @ dz1
        db1 = np.sum(dz1, axis=0, keepdims=True)
        
        return dW1, db1, dW2, db2
    
    def train(self, X_train, y_train, X_val, y_val, epochs=1000, learning_rate=0.01, early_stopping_patience=50):
        """Train regression neural network with validation monitoring"""
        train_losses = []
        val_losses = []
        val_r2_scores = []
        
        best_val_loss = float('inf')
        patience_counter = 0
        best_weights = None
        
        for epoch in range(epochs):
            # Forward pass and training loss
            y_pred_train = self.forward_pass(X_train)
            train_loss = self.mse_loss(y_train, y_pred_train)
            train_losses.append(train_loss)
            
            # Backward pass and weight updates
            dW1, db1, dW2, db2 = self.backward_pass(X_train, y_train)
            
            self.W1 -= learning_rate * dW1
            self.b1 -= learning_rate * db1
            self.W2 -= learning_rate * dW2
            self.b2 -= learning_rate * db2
            
            # Validation evaluation
            y_pred_val = self.forward_pass(X_val)
            val_loss = self.mse_loss(y_val, y_pred_val)
            val_losses.append(val_loss)
            
            val_r2 = self.r2_score(X_val, y_val)
            val_r2_scores.append(val_r2)
            
            # Early stopping logic
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                patience_counter = 0
                best_weights = {
                    'W1': self.W1.copy(), 'b1': self.b1.copy(),
                    'W2': self.W2.copy(), 'b2': self.b2.copy()
                }
            else:
                patience_counter += 1
            
            if epoch % 200 == 0:
                print(f"Epoch {epoch} | Train Loss: {train_loss:.6f} | Val Loss: {val_loss:.6f} | Val R²: {val_r2:.4f}")
            
            if patience_counter >= early_stopping_patience:
                print(f"Early stopping at epoch {epoch}. Best validation loss: {best_val_loss:.6f}")
                self.W1, self.b1 = best_weights['W1'], best_weights['b1']
                self.W2, self.b2 = best_weights['W2'], best_weights['b2']
                break
        
        return {
            'train_losses': train_losses,
            'val_losses': val_losses,
            'val_r2_scores': val_r2_scores
        }
    
    def predict(self, X):
        """Get continuous value predictions"""
        return self.forward_pass(X)
    
    def mse(self, X, y_true):
        """Calculate Mean Squared Error"""
        y_pred = self.predict(X)
        return np.mean((y_true - y_pred) ** 2)
    
    def r2_score(self, X, y_true):
        """
        Calculate R² score (coefficient of determination)
        R² = 1 - (SS_res / SS_tot)
        """
        y_pred = self.predict(X)
        ss_res = np.sum((y_true - y_pred) ** 2)
        ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
        
        # Handle edge case where all predictions are the same
        if ss_tot == 0:
            return 0.0
            
        return 1 - (ss_res / ss_tot)

## Train the model

In [79]:
# network architecture for multi-feature input
input_size = x.shape[1]  # 13 (10 countries + 3 engagement metrics)
hidden_size = 12          # Hidden layer neurons 
output_size = 1          # 1 (total minutes)

print(f"Architecture: {input_size} -> {hidden_size} -> {output_size}")
print(f"Input features: Age + 3 engagement metrics = 4 total")
print(f"Data points: {len(x)}")


Architecture: 4 -> 12 -> 1
Input features: Age + 3 engagement metrics = 4 total
Data points: 5000


In [80]:
# Initialize network 
nn = RegressionNeuralNetwork(input_size, hidden_size, output_size)

# Train the network
training_history = nn.train(x_train, y_train, x_val,y_val, epochs=2000, learning_rate=0.01, early_stopping_patience=100)

# Evaluate performance on all three sets
train_mse = nn.mse(x_train, y_train)
val_mse = nn.mse(x_val,y_val)
test_mse = nn.mse(x_test, y_test)

train_r2 = nn.r2_score(x_train, y_train)
val_r2 = nn.r2_score(x_val,y_val)
test_r2 = nn.r2_score(x_test,y_test)

print(f"\nFinal Result")
print(f"Training MSE: {train_mse:.6f} | R2 {train_r2:.4f}")
print(f"Validation MSE: {val_mse:.6f} | R2: {val_r2:.4f}")
print(f"Test MSE: {test_mse:.6f} | R2: {test_r2:.4f}")

Epoch 0 | Train Loss: 0.701199 | Val Loss: 0.715622 | Val R²: -0.4080
Epoch 200 | Train Loss: 0.508063 | Val Loss: 0.518282 | Val R²: -0.0197
Epoch 400 | Train Loss: 0.500898 | Val Loss: 0.512874 | Val R²: -0.0091
Epoch 600 | Train Loss: 0.498530 | Val Loss: 0.511964 | Val R²: -0.0073
Epoch 800 | Train Loss: 0.497403 | Val Loss: 0.511804 | Val R²: -0.0070
Epoch 1000 | Train Loss: 0.496743 | Val Loss: 0.511773 | Val R²: -0.0069
Epoch 1200 | Train Loss: 0.496295 | Val Loss: 0.511753 | Val R²: -0.0068
Epoch 1400 | Train Loss: 0.495957 | Val Loss: 0.511744 | Val R²: -0.0068
Epoch 1600 | Train Loss: 0.495701 | Val Loss: 0.511717 | Val R²: -0.0068
Epoch 1800 | Train Loss: 0.495503 | Val Loss: 0.511671 | Val R²: -0.0067

Final Result
Training MSE: 0.990669 | R2 0.0027
Validation MSE: 1.023246 | R2: -0.0066
Test MSE: 1.004485 | R2: -0.0020


According to these results, the model learned almost nothing. However, there are some signs of learning as shown below:

- Epoch 0:   Val R²: -0.41 (terrible)
- Epoch 200: Val R²: -0.02 (almost zero)  
- Epoch 1800: Val R²: -0.007 (basically zero)

There is a trend, but it plateaued at near zero.

The training was stable with no wild fluctuations or divergence:
- Epoch 0:   Val R²: -0.41 (terrible)
- Epoch 200: Val R²: -0.02 (almost zero)  
- Epoch 1800: Val R²: -0.007 (basically zero)

There was also no wild fluctuations or divergence
- Training loss: 0.70 → 0.50 (steady decrease)

There could be several reasons why this model is underperforming so drastically. There might not be any meaningful relationships, the learning rate could be too low, or there needs to be more feature engineering.

I am going to try a higher learning rate for this exercise.

In [87]:
# Initialize network 
nn = RegressionNeuralNetwork(input_size, hidden_size, output_size)

# Train the network
training_history = nn.train(x_train, y_train, x_val,y_val, epochs=2000, learning_rate=0.05, early_stopping_patience=300)

# Evaluate performance on all three sets
train_mse = nn.mse(x_train, y_train)
val_mse = nn.mse(x_val,y_val)
test_mse = nn.mse(x_test, y_test)

train_r2 = nn.r2_score(x_train, y_train)
val_r2 = nn.r2_score(x_val,y_val)
test_r2 = nn.r2_score(x_test,y_test)

print(f"\nFinal Result")
print(f"Training MSE: {train_mse:.6f} | R2 {train_r2:.4f}")
print(f"Validation MSE: {val_mse:.6f} | R2: {val_r2:.4f}")
print(f"Test MSE: {test_mse:.6f} | R2: {test_r2:.4f}")

Epoch 0 | Train Loss: 0.634064 | Val Loss: 0.636516 | Val R²: -0.2523
Epoch 200 | Train Loss: 0.497367 | Val Loss: 0.512433 | Val R²: -0.0082
Epoch 400 | Train Loss: 0.496157 | Val Loss: 0.511954 | Val R²: -0.0072
Epoch 600 | Train Loss: 0.495554 | Val Loss: 0.511764 | Val R²: -0.0069
Epoch 800 | Train Loss: 0.495086 | Val Loss: 0.511590 | Val R²: -0.0065
Epoch 1000 | Train Loss: 0.494719 | Val Loss: 0.511444 | Val R²: -0.0062
Epoch 1200 | Train Loss: 0.494374 | Val Loss: 0.511288 | Val R²: -0.0059
Epoch 1400 | Train Loss: 0.494027 | Val Loss: 0.511110 | Val R²: -0.0056
Epoch 1600 | Train Loss: 0.493670 | Val Loss: 0.510894 | Val R²: -0.0052
Epoch 1800 | Train Loss: 0.493202 | Val Loss: 0.510678 | Val R²: -0.0047

Final Result
Training MSE: 0.985601 | R2 0.0078
Validation MSE: 1.021333 | R2: -0.0047
Test MSE: 0.996995 | R2: 0.0054


The results do show improvement:
With a higher learning rate there was faster initial learning as shown below:
- Previous (LR=0.01): Epoch 0: Val R² = -0.41
- Current (LR=0.05):  Epoch 0: Val R² = -0.25  # Better starting point

The new method also shows more learning progress:
- Previous: Train loss: 0.70 → 0.50 (28% decrease)
- Current:  Train loss: 0.63 → 0.49 (22% decrease, but faster)

It is also showing continued improvement:
- Epoch 1800: Val R² = -0.0047 (approaching zero) # Still learning slowly but steadily

However, the core issue persists::
- R² = -0.0047 ≈ 0

The model predictions are essentially the same as just predicting the average (309 minutes) for every user.

So from here it's safe to say we can try something else:
- Possibly try different features
- Try different targets
- Get more data 
- Accept the results

## Make the prediction

In [93]:
# Check for overfitting
if train_r2 - test_r2 > 0.1:
    print("Model may be overfitting (train-test R² gap > 0.1)")
else:
    print("Model shows good generalization")

# Safe access to training history with fallbacks
best_epoch = training_history.get('best_epoch', len(training_history['train_losses']) - 1)
total_epochs = len(training_history['train_losses'])

print(f"\nBest epoch: {best_epoch}")
print(f"Training stopped after {total_epochs} epochs")

# Add training summary
if best_epoch < total_epochs - 1:
    print(f"Early stopping activated (stopped {total_epochs - best_epoch - 1} epochs early)")
else:
    print("Completed all epochs without early stopping")

# Show predictions for different age groups
print("\n" + "="*60)
print("PREDICTIONS BY AGE GROUP")
print("="*60)

# Create test cases for different ages with average engagement
age_groups = [18, 25, 35, 45, 55]
avg_discover = x_raw[:, 1].mean()  # Average discover engagement
avg_songs = x_raw[:, 2].mean()     # Average songs liked
avg_repeat = x_raw[:, 3].mean()    # Average repeat rate

print(f"Using average engagement metrics:")
print(f"Discover Engagement: {avg_discover:.1f}%")
print(f"Songs Liked: {avg_songs:.0f}")
print(f"Repeat Rate: {avg_repeat:.1f}%")

print(f"\n{'Age':<5} {'Predicted Minutes':<18} {'Actual Range':<15}")
print("-" * 45)

for age in age_groups:
    # Create input vector: [age, avg_discover, avg_songs, avg_repeat]
    age_input = np.array([[age, avg_discover, avg_songs, avg_repeat]])
    
    # Scale the input
    age_input_scaled = scaler_features.transform(age_input)
    
    # Predict
    prediction_scaled = nn.predict(age_input_scaled)
    prediction_minutes = scaler_target.inverse_transform(prediction_scaled)[0, 0]
    
    # Find actual range for this age group (±2 years)
    age_mask = (music_data['age'] >= age-2) & (music_data['age'] <= age+2)
    actual_range = music_data[age_mask]['minutes_streamed_per_day']
    
    if len(actual_range) > 0:
        actual_min, actual_max = actual_range.min(), actual_range.max()
        print(f"{age:<5} {prediction_minutes:<18.0f} {actual_min:.0f}-{actual_max:.0f}")
    else:
        print(f"{age:<5} {prediction_minutes:<18.0f} No data")


Model shows good generalization

Best epoch: 1999
Training stopped after 2000 epochs
Completed all epochs without early stopping

PREDICTIONS BY AGE GROUP
Using average engagement metrics:
Discover Engagement: 42.4%
Songs Liked: 50
Repeat Rate: 253.5%

Age   Predicted Minutes  Actual Range   
---------------------------------------------
18    312                10-597
25    307                10-600
35    304                10-599
45    310                11-600
55    316                10-600
